In [1]:
import sys
import os
sys.path.insert(0, os.path.normpath(os.getcwd() + os.sep + os.pardir))

import utils
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.metrics import multilabel_confusion_matrix
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
class WordCountTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def transform(self, X, y=None):
        return [[len(sentence.split())] for sentence in X]
    
    def fit(self, X, y=None):
        return self
    
class CharacterCountTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def transform(self, X, y=None):
        return [[len(sentence)] for sentence in X]
    
    def fit(self, X, y=None):
        return self
    
class StopWordFrequency(BaseEstimator, TransformerMixin):
    def __init__(self, preprocessor=None):
        self.preprocessor = preprocessor
    # check this logic
    def transform(self, X, y=None):
        transformed_x = [self.preprocessor(sentence) for sentence in X]
        return [[(len(X[i])-len(transformed_x[i]))/len(X[i])] if len(X[i]) != 0 else [0] for i in range(len(X))]
    
    def fit(self, X, y=None):
        return self

In [3]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('ngram', CountVectorizer(preprocessor=utils.actual_question_preprocessor)),
        ('word_count', WordCountTransformer()),
        ('character_count', CharacterCountTransformer()),
        ('setnence_remain', StopWordFrequency(preprocessor=utils.actual_question_preprocessor))
    ])),
    ('clf', RandomForestClassifier())
])

In [4]:
actual_question_data = utils.load_data('../data/questions.json', 'actual_question')

In [5]:
pipeline.fit(actual_question_data[:,0], actual_question_data[:,1])

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('ngram',
                                                 CountVectorizer(analyzer='word',
                                                                 binary=False,
                                                                 decode_error='strict',
                                                                 dtype=<class 'numpy.int64'>,
                                                                 encoding='utf-8',
                                                                 input='content',
                                                                 lowercase=True,
                                                                 max_df=1.0,
                                                                 max_features=None,
                                                                 min_df=1,
                       

In [14]:
np.mean(cross_val_score(pipeline, actual_question_data[:,0], actual_question_data[:,1], n_jobs=-1))

0.8901836796145739

In [12]:
param_grid = {
    'features__ngram__ngram_range': [(1,1), (1,2), (1,3)],
    'features__ngram__max_features': [None, 100, 250, 500, 1000, 1500, 2000, 2500, 3000],
    'clf__n_estimators': [100, 250, 500, 750, 1000],
    'clf__max_features': ['auto', 'log2'],
    'clf__max_depth': [None, 100, 500, 1000],
    'clf__min_samples_split': [2, 5, 10, 12],
    'clf__min_samples_leaf': [1, 2, 4, 8],
    'clf__criterion': ['gini', 'entropy'],
    'clf__bootstrap': [True, False]
}

search = RandomizedSearchCV(pipeline, param_grid, n_jobs=-1, n_iter=100)
search.fit(actual_question_data[:,0], actual_question_data[:,1])

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('features',
                                              FeatureUnion(n_jobs=None,
                                                           transformer_list=[('ngram',
                                                                              CountVectorizer(analyzer='word',
                                                                                              binary=False,
                                                                                              decode_error='strict',
                                                                                              dtype=<class 'numpy.int64'>,
                                                                                              encoding='utf-8',
                                                                                              input='content',
     

In [16]:
np.mean(cross_val_score(search.best_estimator_, actual_question_data[:,0], actual_question_data[:,1], n_jobs=-1))

0.9121348991267691

In [14]:
search.best_estimator_

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('ngram',
                                                 CountVectorizer(analyzer='word',
                                                                 binary=False,
                                                                 decode_error='strict',
                                                                 dtype=<class 'numpy.int64'>,
                                                                 encoding='utf-8',
                                                                 input='content',
                                                                 lowercase=True,
                                                                 max_df=1.0,
                                                                 max_features=2000,
                                                                 min_df=1,
                       

In [24]:
utils.actual_question_preprocessor('this is a question')

'   question'

In [10]:
for question in actual_question_data:
    print(question[0])
    print(utils.actual_question_preprocessor(question[0]))
    print()

Questions about Part 1
questions  part numericnumber

Need help with merging from github
   merging  sys

Trouble importing my previous data structures
trouble importing  previous data structures

I need help with the iterator right now
     iterator right 

We're still having trouble running the Makefile
 still  trouble running  sys

I need help figuring out what is wrong with dfs.
   figuring    wrong  dfs

when theres double letters doesnt work
 theres double letters doesnt work

longest word not returning
longest word  returning

Program runs infinitely. Tried adjusting iterator.
program runs infinitely tried adjusting iterator

argh hi this is really hard
argh hi   really hard

Having trouble transitioning from VM to GitLab.
 trouble transitioning  sys  sys

do not know what is wrong  rn
  know   wrong rn

doint know how to do recursive stuff
doint know    recursive 

what is a hashcode?
   hashcode

Stuck on task 2
stuck  task numericnumber

what am i doing
   

does order matter

 linear coalescing  giving   errorcode fault

How would we create a test for vector.c? 
 would  create  test  filename


Need debugging help with Test B. Tried intellj debugger
 debugging   test b tried intellj debugger

Help with git clone and passphrase
  sys clone  passphrase

Have trouble with lab
 trouble  lab

i'm confused about func4
im confused  func4

No such element exception is confusing me and im not sure where it's coming from.
  element errorcode  confusing   im  sure   coming 

failing on assert is_pool_address in part 3
failing  assert camelsnakecase  part numericnumber

                _,,......_
                 ,-'          `'--.
              ,-'  _              '-.
     (`.    ,'   ,  `-.              `.
      \ \  -    / )    \               \
       `\`-^^^, )/      |     /         :
         )^ ^ ^V/            /          '.
         |      )            |           `.
         9   9 /,--,\    |._:`         .._`.
         |    /   /  `.  \    `.      (   `.`.
   

 100% certain   understand  camelsnakecase 

      _.---._    /\\
    ./'       "--`\//
  ./              o \          .-----.
 /./\  )______   \__ \        ( help! )
./  / /\ \   | \ \  \ \       /`-----'
   / /  \ \  | |\ \  \7--- ooo ooo ooo ooo ooo ooo
_ --- _ /\\ / --\// /  \ ----- / /\ camelsnakecase \__ \  / / /\ \ | \ \ \ \ /----- / / \ \ | |\ \ \7--- ooo ooo ooo ooo ooo ooo

Program segfaults on malloc() in the fit step, gdb says the free blocks of memory don't have proper prev/next pointers.
Also memory is getting garbled somehow. That might be the cause of the other issue.
program errorcode  func   fit step gdb says  free blocks  memory dont  proper prev next pointers also memory  getting garbled somehow  might   cause    issue

I'm not sure how to implement the heap checker
im  sure   implement  heap checker

Gitlab cant make my code even though I can. Also, debugging for tests. 
sys cant make  code even though   also debugging  tests

My checkheap isnt working. Even in ess

   know     nonnumericnumber error   encodetext also    undersatnd part  

Regarding an error message(main is usually not a non-static function)
regarding  error message main  usually   non-static function

peter help send some help please i love you
name  send     love 

I'm getting a stack buffer overflow error when I run test7 and I'm not sure why
im getting  stack buffer errorcode error   run test7  im  sure 

Am I supposed to loop through the file of word everytime I create a new random hangman chooser?
  supposed  loop   file  word everytime  create  new random hangman chooser

I tried to test wc.c on dictionary.txt and have the following problems:
wc.c:14:18: warning: implicitly declaring library function 'malloc' with type 'void *(unsigned long)'
      [-Wimplicit-function-declaration]
  char *result = malloc(number_to_read * sizeof(char));
                 ^
wc.c:14:18: note: include the header <stdlib.h> or explicitly provide a declaration for 'malloc'
wc.c:18:5: warning: imp

getting camelsnakecase error  iterators  cant seem  figure        suspect   probably  iterator  continuously looping   next element   stopping  

Currently, the Dijkstra tests are reporting that they are finding longer paths than they are supposed to, or not at all. I have tried running the debugger on the Dijkstra algorithm and checked my minFourHeap, but no immediately apparent problem appears.
currently  dijkstra tests  reporting    finding longer paths    supposed        tried running  debugger   dijkstra algorithm  checked  camelsnakecase   immediately apparent  appears

need fix
 fix

How can we determine if the file "f" has more characters and if the character read is a newline? My first inclination was to use the fef() method, however we this would not allow for the reading of two characters at a time (as in the case of a newline)
   determine   file f   characters    character read   newline  first inclination   use  func method however   would  allow   reading  nonnumericnumb

In [15]:
res = []
with open('../data/questions.json', 'r') as f:
        res = np.array([utils.remove_non_ascii(eval(data_point)['question']) for data_point in f.read().splitlines()])

In [18]:
for question in res:
    if search.best_estimator_.predict([question]) == 't':
        print(question)
        print()

All sorts of issues with peekFront

neighbors being weird in beaver map graph

Need help with merging from github

Unsure about some thread functions

trying to get star to work


Question about remove method implementation

Not pushing to git.

questions about segregated storage

"error: typedef redefinition with different types"

Histogram has a lot of noise, causing errors.

Where do I start with garbage collector?

size isn't updating correctly in ChaininHashDictionary

Having trouble writing the randomly initialized key 

Several quick conceptual questions.

NullPointerException on add() functions

Clarification on stack frame question

Dikstra Doesnt work on map

Dijkstra doesn't work on map

graph implementation does not work

MinFourHeap and conceptual ngrammap

help on lab- error with deserialization

BST Dictionary debugging

MoveToFront dictionary not working

confirming ideas about spec about spec about spec mgjrieafgjf
 erg
grjgierbs
t
n
nr
s
n
sfn
ts
hte
hejhtsehjkel;


I

I need help ensuring all methods are implemented properly


Problem with getting rsp location.

oh no :( we are failing something weird

Asking about assembly code in math.annotated.s

Git tests are pending

Just confused about how movetodictionary works

a tests confusion about algorithm

I need help with understandingwhat getSolution is doing

stress tests, assertion failed errors

project 5 error caused by ArrayDeque?

What does hashcode do

ArrayDeque class- issues with peek running constant time 

My LinkedDeque is completely broken.

Linked deque iterator implementation 

ChainingHashDictionary tests not running

I need help debugging the edit distances method

Types in TrieMap AutoCompleter for complete()

Restructuring the way we iterate through our data

can I get some help understanding "puts"

Need help understanding __sthread_initialize_context(stack, f, arg)

Footer issues not matching with size.

confused on working with ssh machine

Confused about using function in threa

Code doesn't push or commit

Existing key issues with the size, possibly more once those run

understanding references/values in garbage collector

cant find source of null pointer exception

different behavior when running through valgrind vs running normally

My contains key complexity doesn't pass. It's linear time

take constant time problem still not fixed
iterator doesn't work

CLOCK-PRO general question about the different hands

I started project 00 today because of 

implementing Graph class conceptual questions

Put and get are acting weird                      

Understanding the given code. 

phase 3 not working "Misfire: You called touch3 (" ") "


"data/empty"  test returning execution error

MoveToFrontDictionary cant figure out which method is wrong 

shifted a lot of code into helper methods, used debugger

We don't know how to construct a minfourheap.

confusion regarding function being called in phase 4

NullPointerException weirdness on A tests

Issues with my mount

Our remove method doesn't work 

Can't seem to get number of guesses remaining correctly.

I have corrupted data in my garbage collector

We're a bit lost on how TrieMap works

Printing of the assembly code by our functions.

In SubstitutionCipher, I have a question about 

Structuring put so as to keep track of TrieNodes

jumping to touch 1 instead of touch 2

Our b tests are stuck in a loop

Works from (0,0) to (0,7) breaks at (0,8)

Doubly linked list errors, specifically nullpointer exceptions

Question about initializing PTE in map function.

Almost except one test... checked conceptually with friends

We are having issues with removing stuff for the MoveFrontDictionary

What does not included: null mean?

We're having trouble importing the vector.h file.

I am having trouble constructing my dequeue

import and package statements have red squigglies

Need help with getting my partner on atom with me

weird bug for recursive stop and copy

confusion about strings_not_equal function

Part 1, need help initializing array



lseek function explanation and parts a and b

need help on one of teh optimizations in the pdf

JSON error in constructor for beaver graph

Hashtable + table implementation just checking 

Segmentation fault in meltdown not caught by signal handler

I'm afraid I'm misunderstanding how to write hashcode()

percolate up and down returning wrong values

How to get key from value

How to use assembler

Usage of semaphores to solve the concurrency problem

Update methods in arraydeque not updating properly, bad public interface

Floating point exception error with my stage 5 

I am having issues with having the right values. I think I may have initialized tail wrong?

to string illegal start of type error

I need help working on make guess to check the longest value

Size of node_t so can find offset of value from start of value in num_node_t.

Question about importing classes

semaphore function issue; never seems to malloc

Getting seg fault on user

...

LONGEST.C:  I don't understand how to access characters in the file and copy them.  Only how to read them, which to my knowledge doesn't return anything useful.  Also, what are we supposed to return when there are two long words of equal size.

I am getting a UnsupportedOperationException when trying to modify a key in the SubstitutionCipherSolver. It says that the key is an immutable collection but I am confused because I modified it (assigned a key to a new value) without problem in an earlier method. 

One of the ArrayDeque time tests is failing constantly and I'm not sure about how the circular fixed array should work.

For the BST Dictionary, I am getting a stack overflow error on the smoke tests, and I can't figure out why.

Trying to understand how the interfaces relate to the code in the C tests. More specifically, how to encode if the current instance is a stack or a queue. 

I'm not sure how to use double pointers in mystring.c. Asan doesn't report errors but I am certain my 

I keep getting an error with initializing a final field in the constructor and noSuchElementException

LinkedDeque has 37/43 tests passing. My contains() tests are not working, I believe an index may be off by 1 somewhere.

I've written the code for wc.c and it gets the correct numbers. However, whenever I run the tests, it says "Execution Error" for all of the tests except for the empty file, which I pass. I don't know how to fix this issue. Code is pushed.

I think I'm misunderstanding something with the indices or size or something. I've tried printing the lists and checking to make sure that I'm incrementing/decrementing my size as necessary, but I'm still failing tests and I"m not sure where else to look

I'm able to test my wc.c program from the command line and run it successfully, but when I run the tests it says there's an "Execution Error," which I cant seem to find the source of. I've tried looking through the test script and it doesn't specify what an Execution Error entail

I tried to test wc.c on dictionary.txt and have the following problems:
wc.c:14:18: warning: implicitly declaring library function 'malloc' with type 'void *(unsigned long)'
      [-Wimplicit-function-declaration]
  char *result = malloc(number_to_read * sizeof(char));
                 ^
wc.c:14:18: note: include the header <stdlib.h> or explicitly provide a declaration for 'malloc'
wc.c:18:5: warning: implicit declaration of function 'free' is invalid in C99 [-Wimplicit-function-declaration]
    free(result);
    ^
wc.c:24:18: warning: result of comparison against a string literal is unspecified (use strncmp instead)
      [-Wstring-compare]
      if (result == "\n") {
                 ^  ~~~~
3 warnings generated.
/usr/bin/ld:data/dictionary.txt: file format not recognized; treating as linker script
/usr/bin/ld:data/dictionary.txt:2: syntax error
clang-5.0.1: error: linker command failed with exit code 1 (use -v to see invocation)

My code is returning an inverted array of the array 

problems with making field with stored word final

I am not sure the best way to approach the evil hangman chooser, specifically how to implement the division of words into sets based on their pattern.

Wc.c compiles but says there is a memory leak and I cannot find it.

My coalescing is not working, and I've spent a ton of time on it, and am not sure why...

Getting a segfault when returning for sthread_restore, after popping all values into registers 

Running buf phase 2. Am getting illegal instruction when I try to run the exploit string. The memory address the code should jump to is the start of my exploit string, which has the code for moving my cookie to %rdi and then pushes the address of touch2 to the stack and returns it. I am pretty sure that the code isn't broken, and I know the nonsense byte to overrun buf are long enough.

I'm not sure how to apply the use of a random call to a set, or why a set would be the best choice to use for this purpose anyway.

There's an error th

Project 8 with a late token. The C tests are almost working, but the "Nodes and Edges" test fails with a IllegalArgumentException in my neighbors() function (i.e. it calls for an vertex that apparently does not exist in )

The CMS server is taking a really long time to put files on atom. Just checking if there is a current issue? I put the command atom longest.c and it hasn't done anything for like 3 min.

When testing for fibonacci, it seems like mine just copies the given value to register 2 but I wasn't sure why it was doing that?

Project 8 with a late token. The C tests are almost working, but the "Nodes and Edges" test fails with a IllegalArgumentException in my neighbors() function (i.e. it calls for an vertex that apparently does not exist in my )

I created some tests in my Main for addFront/addBack and they work as expected when I add a small number of elements, but they are not passing the LinkedDeque tests.

My removeBack is not working and not sure why, currently debugging

In [19]:
for question in res:
    if search.best_estimator_.predict([question]) == 'f':
        print(question)
        print()

Can't find why NoSuchElement 

Confused on what rbp does

do i need this while loop assembly



Questions about Part 1

questions about part 1

not passing enqueue/dequeue

Plaintext says it is wrong

we need help working on the ctests.

test common not working

stress test not working?

Clarity for B tests

Confused about how getguesses works

help on vector and testing


dictionary not static

I fail all the tests

enqueue and dequeue methods

My C tests aren't working...

questions about the chainingHashDictionary

How to approach getCountsAfter()

how to write casdfasdfasdf

how to start movetofrontdictionary

How to handle getWord

stuck on Linked deque

My remove is broken

My git is

I'm confused about where to start.

Aging policy is underperforming

My Iterator isn't working

Really confused about what lock does

stack overflow for smoke

Dictionary Questions

project 4 C test iterator 

I need help with the iterator right now

my tests won't run


my tests wont run


Failing 

C tests help with the dequeue

part 3, 4 and 5

Need o be checked off for lab

Signal handling not working

how does TrieMap work?

how to recurse? pt. 2

How to approach phase 1


How to approach phase 2


Passing recursive test

a tests conceptual, c tests

project 7 a tests

b tests project 4

A tests Project 8

B tests not passing

not passing marvel tests

C tests not passing

Complexity tests not passing. 

Probably will have some issue with BST

quick question on syntax

Debugging for increase key


size of each thing in data

Help with percolation and enqueuing 


get method on C tests

What do tests test?

need help understanding buf

Need help debugging my code 

Questions about some 

one test fail for arraydeuque

Getting odd error

I am starting the project now

Constant time for linked list

error on my circular array 

LinkedDeque is confusing me


need adam's help on smth old not working

not understanding priority

ask for partners' code

Establishing code for beaverGr

A tests help with memoization 

Help with writing A tests

Help with A tests algorithm.

B tests help with getWordsAfter

A tests help with dijkstra

Working on graph

If I include "

lots of bugs in B tests

What is init for?

dfs need help this pls not work

BeaverMapGraphs is confusing

CircularFixedSize... is broken.

bug in explicit free list

atom not saving file

type error in getWordsAfter

Having issues with BST remove

Implementation of explicit allocator

can't figure out asan error



Debug heap-buffer-overflow

Debugging coalescing seg fault

Help debugging my MoveToFrontDictionary

Stress test takes forever

working on getCompletion

Working on FrontDictionary

working on PopulateTitles

Working on tail

Working on Ipriorityqueue 

polygon.c errors with pointers

Value is 

CS 2 how to get started 

Something's wrong with my code I think

How do I play a game?

Not sure why 


Time complexity test errors.

locked undefined + test

understanding plot for q9

debugging NO M

MoveToFront and NGramsTest

percolateUp and percolateDown

B tests...spec unclear

Not sure why we're failing

i dont know whats wrong please help lmao; also confused about hashing


need help debugging right now

Size methods off by one

Cannot get thing open

Debugging vector.c 

longest.c debugging 

Passed Locals failing MoreLocals fhdhdjhdhdhshshsjshshshjsjsjshshshjsjsHdhsjsbsjsjjsbsjsjsjshsjsjsjjshsjsjsjshhdhddd

question about coalescing

how to add  to the head

Struggling with Part 2














percolate up giving nulll 

help with A tests please

how to do registers for gcd

cache level 1 2 3 

Help with debugging addedge

getCompletions() for TrieMap 

methods in linkeddeque

what to doooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo

Off by indices in SortTester

Errors with task 1?

Errors with task 2?

Test failing, unsure why

Help with the ChainHashDictionary.

What is @Override and @Test and stuff?

How to operate VAR and LET


We are confused

My IDeque 

Problem with porting forward classes.

problems with Dijkstra

Failing to

debugging something - sujai

Help with Iterator

#3.c


debugging lab 1                        

problem with starting meltdown image on virtualbox:

VT-x/AMD-V hardware acceleration is not available on your system. Your 64-bit guest will fail to detect a 64-bit CPU and will not be able to boot.



alpha beta initial values

Help with Circular Array.


Confused about TrieMap structure

2 values possible??????

iterator, hash 

help help help help help help 

tried modifying stuff

exiting the 

Various issues in MoveToFront

I require assistance with debugging.

null null null null 



having trouble with moveup/movedown

Having trouble with BeaverMapTests

My TrieMap ite

Doing phase 1

Writing in file

b tests help please 

help a tests please

A tests help please

dijkstra tests help please

a tests help please 

help b tests please

help c tests please

sorted set index qeustion 

I don't know wh

      _ . - - - . _    /\\
    ./'        " - - ` \//
  ./              o \          .-----.
 /./\  )______   \__ \        ( help! )
./  / /\ \   | \ \  \ \       /`-----'
   / /  \ \  | |\ \  \7--- ooo ooo ooo ooo ooo ooo

      _ . - - -    . _ /\\
    ./'        " - -    `\//
  ./              o \          .-----.
 /./\  )______   \__ \        ( help! )
./  / /\ \   | \ \  \ \       /`-----'
   / /  \ \  | |\ \  \7--- ooo ooo ooo ooo ooo ooo

      _ . - - - . _    /\\
    ./'        " - - `   \//
  ./              o \          .-----.
 /./\  )______   \__ \        ( help! )
./  / /\ \   | \ \  \ \       /`-----'
   / /  \ \  | |\ \  \7--- ooo ooo ooo ooo ooo ooo

We are confused.

movetofrontdictionary, put()

ListNode etc.

ChainingHashDictionary put

make file dumbass

victor, I need help

victor I need help

null



Guitar pluck question

virtual memory question


Help we broke long

CLOCK/LRU policy?

return statement 

void * stuff



recursive call question

"bad register nam

In [35]:
search.best_estimator_.predict_proba(['Confused about how getGuesses() works'])

array([[0.49751058, 0.50248942]])